# Talktorial 8

# Protein data acquisition: Protein Data Bank (PDB)

#### Developed in the CADD seminars 2017 and 2018, AG Volkamer, Charité/FU Berlin 

Anja Georgi, Majid Vafadar and Dominique Sydow

## Aim of this talktorial

In this talktorial, we conduct the groundwork for the next talktorial where we will generate a ligand-based ensemble pharmacophore for EGFR. Therefore, we 
(i) fetch all PDB IDs for EGFR from the PDB database, 
(ii) retrieve five protein-ligand structures, which have the best structural quality and are derived from X-ray crystallography, and 
(iii) align all structures to each in 3D as well as extract and save the ligands to be used in the next talktorial.

## Learning Goals

### Theory
* Protein Data Bank (PDB)
* Python package PyPDB
 
### Practical

* Select query protein
* Get statistic on PDB entries for query protein
* Get all PDB IDs for query protein
* Get meta information on PDB entries
* Filter and sort meta information on PDB entries
* Get meta information of ligands from top structures
* Draw top ligand molecules
* Create protein-ligand ID pairs
* Get the PDB structure files
* Align PDB structures
 
## References

* Protein Data Bank 
([PDB website](http://www.rcsb.org/pdb>))
* PyPDB python package 
([<i>Bioinformatics</i> (2016), <b>32</b>, 159-60](https://academic.oup.com/bioinformatics/article-lookup/doi/10.1093/bioinformatics/btv543))
* PyPDB python package documentation 
([PyPDB website](http://www.wgilpin.com/pypdb_docs/html/))
* PyMol selection algebra 
([PyMolWiki: selection algebra](https://pymolwiki.org/index.php/Selection_Algebra))

## Theory

### Protein Data Bank (PDB)

The Protein Data Bank (PDB) is one of the most comprehensive structural biology information databases and a key resource in areas of structural biology, such as structural genomics and drug design. ([PDB website](http://www.rcsb.org/pdb>))

Structural data is generated from structural determination methods such as X-ray crystallography (most common method), nuclear magnetic resonance (NMR), and cryo electron microscopy (cryo-EM). 
For each entry, the database contains (i) the 3D coordinates of the atoms and the bonds connecting these atoms for proteins, ligand, cofactors, water molecules, and ions, as well as (ii) meta information on the structural data such as the PDB ID, the authors, the deposition date, the structural determination method used and the structural resolution.

The structural resolution is a measure of the quality of the data that has been collected and has the unit Å (Angström). The lower the value, the higher the quality of the structure. 

The PDB website offers the 3D visualization of the protein structures (with ligand interactions if available) and the structure quality metrics, as can be seen for the PDB entry of an example epidermal growth factor receptor (EGFR) with the [PDB ID 3UG5](https://www.rcsb.org/structure/3UG5).

<img src="./images/protein-ligand-complex.png" align="above" alt="Image cannot be shown" width="400">
<div align="center"> Figure 1: The protein structure (in gray) with an interacting ligand (in green) is shown for an example epidermal growth factor receptor (EGFR) with the PDB ID 3UG5 (figure by Dominique Sydow).</div>

### PyPDB

PyPDB is a python programming interface for the PDB and works exclusively in Python 3. 
This package facilitates the integration of automatic PDB searches within bioinformatics workflows and simplifies the process of performing multiple searches based on the results of existing searches. 
It also allows an advanced querying of information on PDB entries. 
The PDB currently uses a RESTful API that allows for the retrieval of information via standard HTML vocabulary. PyPDB converts these objects into XML strings. 
([<i>Bioinformatics</i> (2016), <b>32</b>, 159-60](https://academic.oup.com/bioinformatics/article-lookup/doi/10.1093/bioinformatics/btv543))

A list of functions is provided on the PyPDB documentation website ([PyPDB website](http://www.wgilpin.com/pypdb_docs/html/)).

## Practical



In [3]:
# Import necessary libraries
from pypdb import *
from pymol import *

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
IPythonConsole.ipython_useSVG=True

import pprint
import glob

import pandas as pd
from array import array
import numpy as np
import collections

import matplotlib.pyplot as plt
%matplotlib inline

### Select query protein

We use EGFR as query protein for this talktorial. The UniProt ID of EGFR is `P00533`, which will be used in the following to query the PDB database.

### Get statistic on PDB entries for query protein 

First, we ask the question: How many PDB entries are deposited in the PDB for EGFR per year and how many in total?

We can do a search on the [PDB website](http://www.rcsb.org/pdb>) with the search term `P00533`. 
In October 2018, the PDB returned 179 search results.

Using `pypdb`, we can find all deposition dates of EGFR structures from the PDB database. The number of deposited structures is needed to set the parameter `max_results` of the function `find_dates`.

In [4]:
# Note: Parameter max_results default is 100, which is too low for EGFR
# If max_results > maximal number of EGFR structures: error, 
# Therefore we checked beforehand how many results exist (#179)

# This database query may take a moment (minute to couple of minutes)
all_dates = find_dates("P00533", max_results=179)  

NameError: name 'find_dates' is not defined

In [ ]:
print("Number of EGFR structures found: " + str(len(all_dates)))

In [ ]:
# Example of deposition dates
all_dates[:3]

We extract the year from the deposition dates and calculate a depositions-per-year histogram.

In [5]:
# Extract year
all_dates = np.asarray(all_dates)
all_years = np.asarray([int(depdate[:4]) for depdate in all_dates])

# Calculate histogram
bins = max(all_years)-min(all_years)  # Bin number = year range
subs_v_time = np.histogram(all_years, bins)

# All entries (excluding 2018) are plotted
dates, num_entries = subs_v_time[1][:-1], subs_v_time[0]  

# Show histogram
fig = plt.figure()
ax = plt.subplot(111)
ax.fill_between(dates, 0, num_entries)
ax.set_ylabel("New entries per year")
ax.set_xlabel("Year")
ax.set_title("PDB entries for EGFR")
plt.show()

NameError: name 'all_dates' is not defined

### Get all PDB IDs for query protein

Now, we get all PDB structures for our query protein EGFR, using the `pypdb` function `make_query` and `do_search`.

In [ ]:
search_dict = make_query("P00533")  # May run into timeout when max_results is 180 or more
found_pdb_ids = do_search(search_dict)

print("PDB IDs found for query: ")
print(found_pdb_ids)

print("\nNumber of structures: " + str(len(found_pdb_ids)))

### Get meta information for PDB entries

We use `describe_pdb` to get meta information about the structures, which is stored per structure as dictionary.

Note: we only fetch meta information on PDB structures here, we do not fetch the structures (3D coordinates), yet.

In [ ]:
# This database query may take a moment
pdbs = []
for i in found_pdb_ids:
  pdbs.append(describe_pdb(i))

pdbs[0]

### Filter and sort meta information on PDB entries

Since we want to use the information to filter for relevant PDB structures, we convert the data set from dictionary to DataFrame for easier handling.

In [ ]:
pdbs = pd.DataFrame(pdbs)
pdbs.head()

In [ ]:
print("Number of PDB structures for EGFR: " + str(len(pdbs)))

We start filtering our dataset based on the following criteria:

#### 1. Experimental method: X-ray diffraction

We only keep structures resolved by `X-RAY DIFFRACTION`, the most commonly used structure determination method. 

In [ ]:
pdbs = pdbs[pdbs.expMethod =="X-RAY DIFFRACTION"]
print("Number of PDB structures for EGFR from X-ray: " + str(len(pdbs)))

#### 2. Structural resolution

We only keep structures with a resolution equal or lower than 3 Å (Angström). The lower the resolution value, the higher is the quality of the structure (= the higher is the certainty that the assigned 3D coordinates of the atoms are correct). Below 3 Å, atomic orientations can be determined  and therefore is often used as threshold for structures relevant for structure-based drug design.

In [ ]:
pdbs_resolution = [float(i) for i in pdbs.resolution.tolist()]
pdbs = pdbs[[i <= 3.0 for i in pdbs_resolution]]

print("Number of PDB structures for EGFR from X-ray with resolution <= 3.0 Angström: " + str(len(pdbs)))

We sort the data set by the structural resolution. 

In [ ]:
pdbs = pdbs.sort_values(["resolution"], 
                        ascending=True, 
                        na_position='last')

We check the top PDB structures (sorted by resolution): 

In [ ]:
pdbs.head()[["structureId", "resolution"]]

#### 3. Ligand-bound structures

Since we will create ensemble ligand-based pharmacophores in the next talktorial, we remove all PDB structures from our DataFrame, which do not contain a bound ligand: we use the `pypdb` function `get_ligands` to check/retrieve the ligand(s) from a PDB structure. PDB-annotated ligands can be ligands, cofactors, but also solvents and ions. In order to filter only ligand-bound structures, we (i) remove all structures without any annotated ligand and (ii) remove all structures that do not contain any ligands with a molecular weight (MW) greater than 100 Da (Dalton), since many solvents and ions weight less. Note: this is a simple, but not comprehensive exclusion of solvents and ions. 

In [ ]:
# Get all PDB IDs from DataFrame
pdb_ids = pdbs["structureId"].get_values().tolist()

In [ ]:
# Remove structures 
# (i) without ligand and 
# (ii) without any ligands with molecular weight (MW) greater than 100 Da (Dalton)

mw_cutoff = 100.0  # Molecular weight cutoff in Da

# This database query may take a moment
removed_pdb_ids = []
for i in pdb_ids:
    ligand_dict = get_ligands(i)
    
    # (i) Remove structure if no ligand present
    if ligand_dict["ligandInfo"] is None:
        pdb_ids.remove(i) # Remove ligand-free PDB IDs from list
        removed_pdb_ids.append(i) # Store ligand-free PDB IDs
    
    # (ii) Remove structure if not a single annotated ligand has a MW above mw_cutoff
    else:
        # Get ligand information
        ligs = ligand_dict["ligandInfo"]["ligand"]
        # Technicality: if only one ligand, cast dict to list (for the subsequent list comprehension)
        if type(ligs) == dict:
            ligs = [ligs]
        # Get MW per annotated ligand
        mw_list = [float(i["@molecularWeight"]) for i in ligs]
        # Remove structure if not a single annotated ligand has a MW above mw_cutoff
        if sum([mw > mw_cutoff for mw in mw_list]) == 0:
            pdb_ids.remove(i) # Remove ligand-free PDB IDs from list
            removed_pdb_ids.append(i) # Store ligand-free PDB IDs

print("PDB structures without a ligand (removed from our data set):")
print(removed_pdb_ids)

In [ ]:
print("Number of structures with ligand: " + str(len(pdb_ids)))

### Get meta information of ligands from top structures

In the next talktorial, we will build ligand-based ensemble pharmacophores from the top `top_num` structures with the highest resolution.

In [ ]:
top_num = 4  # Number of top structures
pdb_ids = pdb_ids[0:top_num]
pdb_ids

We fetch the PDB information about the top `top_num` ligands using `get_ligands`, to be stored as *csv* file (as dictionary per ligand).

If a structure contains several ligands, we select the largest ligand. Note: this is a simple, but not comprehensive method to select ligand binding the binding site of a protein. This approach may also select a cofactor bound to the protein. Therefore, please check the automatically selected top ligands in PyMol for further usage.

In [ ]:
ligands_list = []

for i in pdb_ids:
    ligands = get_ligands(i)["ligandInfo"]["ligand"]
    # Technicality: if only one ligand, cast dict to list (for the subsequent list comprehension)
    if type(ligands) == dict:
        ligands = [ligands]

    weight = 0
    this_lig = {}
    
    # If several ligands contained, take largest
    for lig in ligands:
        if float(lig["@molecularWeight"]) > weight:
            this_lig = lig
            weight = float(lig["@molecularWeight"])
            
    ligands_list.append(this_lig)

# Change the format to DataFrame
ligs = pd.DataFrame(ligands_list)
ligs

In [ ]:
ligs.to_csv('../data/T8/PDB_top_ligands.csv', header=True, index=False, sep='\t')

### Draw top ligand molecules

In [ ]:
PandasTools.AddMoleculeColumnToFrame(ligs, 'smiles')
Draw.MolsToGridImage(mols=list(ligs.ROMol), 
                     legends=list(ligs['@chemicalID']+', '+ligs['@structureId']), 
                     molsPerRow=top_num)

### Create protein-ligand ID pairs

In [ ]:
pairs = collections.OrderedDict()

for idx, row in ligs.iterrows():
    pairs[str(row['@structureId'])] = str(row['@chemicalID'])

print(pairs)

### Get the PDB structure files

We now fetch the PDB structure files, i.e. 3D coordinates of the protein, ligand (and if available other atomic or molecular entities such as cofactors, water molecules, and ions) from the PDB using the `pypdb` function `get_pdb_file`. 
Available file formats are *pdb* and *cif*, which store the 3D coordinations of atoms of the protein (and ligand, cofactors, water molecules, and ions) as well as information on bonds between atoms. Here, we work with *pdb* files.

In [ ]:
# Fetch pdb file and save locally
for prot, lig in pairs.items():
    pdb_file = get_pdb_file(prot, filetype='pdb', compression=False)
    with open('../data/T8/'+ prot + '.pdb', 'w') as f:
        f.write(pdb_file)

### Align PDB structures

Since we want to build ligand-based ensemble pharmacophores in the next talktorial, it is necessary to align all structures to each other in 3D. We will use the molecular visualization program PyMol for this task, which can also be used from within the Jupyter notebook. PyMol aligns two structures at a time in a way that the distance of atoms between the two structures is minimized.

First, we will launch PyMol from the command line (in quiet mode, i.e. the GUI will not open).

In [ ]:
# Launch PyMol in quiet mode
pymol.pymol_argv = ['pymol','-qc']
pymol.finish_launching()

For the alignment, we choose a reference structure file (immobile PDB, 'target') onto which the other ('query') structure files are superimposed using the PyMol command `cmd.align(query, target)`. All `cmd.` commands are commands for PyMol.

We save the aligned structures with the new coordinates as *pdb* files. We also extract the ligand from the structure file and save it separately as *pdb* file to be used in the next talktorial.

In [ ]:
# Save alignment logs to file
f = open("../data/T8/alignments.log", "w")

# Variable distinguishing between immobile and mobile structure during alignment
immobile_pdb = True
refAlignTarget='non'
refAlignQuery='non'

# Align proteins on first protein
for prot, lig in pairs.items():
    
    # Immobile structure (reference structure for alignment)
    if immobile_pdb:
        target = prot
        f.write('Immobile target: ' + prot + '\n')
        
        # Load pdb file (complex of protein and ligand)
        targetFile = cmd.load('../data/T8/' + target + '.pdb')
        # Store name for refined alignment
        refAlignTarget='('+target+' within 5 of resn '+lig+')'
        
        # Save complex as pdb file
        cmd.save('../data/T8/' + target + '_algn.pdb', selection=target)
        
        # Select only the ligand with the selected name
        ligObj = cmd.select('ligand', target + ' and resn ' + lig)
        # Save selection as pdb file
        cmd.save('../data/T8/' + target + '_lig.pdb', selection='ligand', format='pdb')
        # Delete ligand selection
        cmd.delete(ligObj)
        # Target selected
        immobile_pdb = False
        
    # Mobile structures (which are aligned to reference structure)
    else:
        query = prot
        f.write('-- align %s to %s \n' %(query, target))
        
        # Load pdb file (complex of protein and ligand)
        queryFile = cmd.load('../data/T8/' + query + '.pdb')
        
        # Align structures (proteins) with focus on binding site
        refAlignQuery= '('+query+' within 5 of resn '+lig+')' 
        values = cmd.align(refAlignQuery, refAlignTarget)
        
        
        # If structures cannot be aligned (i.e. if RMSD > 5A), skip alignment
        if values[0] > 5:
            f.write('--- bad alignment: skip structure\n')
        else:
            # Save complex as pdb file
            cmd.save('../data/T8/' + query + '_algn.pdb', selection=query)
            
            # Select only the ligand
            ligObj = cmd.select('ligand', query + ' and resn ' + lig)
            
            # Save selection as pdb file
            cmd.save('../data/T8/' + query + '_lig.pdb', selection='ligand', format='pdb')
            
            # Delete ligand selection
            cmd.delete(ligObj)
            
        # Delete "query" selection
        cmd.delete(query)
    
# Delete "target" selection
cmd.delete(target)

f.close()

We quit the PyMol application.

In [ ]:
# Quit PyMol
pymol.cmd.quit()

We check the existence of all ligand *pdb* files. If files are missing, please check the protein-ligand structures by hand in PyMol.

In [ ]:
mol_files = []
for file in glob.glob("../data/T8/*_lig.pdb"):
    mol_files.append(file)
mol_files

## Discussion

In this talktorial, we learned how to retrieve protein and ligand meta information and structural information from the PDB. We retained only X-ray structures and filtered our data by resolution and ligand availability. Ultimately, we aimed for an aligned set of ligands to be used in the next talktorial for the generation of ligand-based ensemble pharmacophores. 

In order to enrich information about ligands for pharmacophore modeling, it is advisable to not only filter by PDB structure resolution, but also to check for ligand diversity (see **talktorial 5** on molecule clustering by similarity) and to check for ligand activity (i.e. to include only potent ligands). 

## Quiz

1. Summarize the kind of data that the Protein Data Bank contains.
2. Explain what the resolution of a structure stands for and how and why we filter for it in this talktorial.
3. Explain what an alignment of structures means and discuss the alignment performed in this talktorial.